In [1]:
import pandas as pd
import numpy as np
import ast
import re
import unicodedata
import sacrebleu

In [2]:
df = pd.read_csv('Data.csv')

In [3]:
df_new = df[df.notna().all(axis=1)]

## 1.
B1: Chuyển về chữ thường để tính bleu ko lỗi (done)  

B2 :Kiểm tra trùng hoàn toàn và xử lý:  (xét theo ministry)  

    + Xử lý các dòng trùng nhau hoàn toàn (chỉ giữ lại 1)  (done)
    + Xử lý các dòng có câu hỏi trùng nhưng câu trả lời khác nhau ( ưu tiên câu trả lời có TTHC đầy đủ hơn, nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi)  

### Vấn đề 1: 
#### Chuẩn hóa kí tự để so sánh bleu score
- Loại bỏ in hoa
- Không trùng khớp giữa định dạng chữ tiếng Việt có dấu, thống nhất NFC(Normalization Form C) 1 dạng chuẩn hóa unicode phổ biến trong tiếng Việt
- Loại bỏ các kí tự dấu câu, chỉ giữ lại từ và ngày tháng
- Đánh dấu các kí tự tiếng việt
- Loại bỏ khoảng trắng thừa

In [4]:
#df_new = df_new.assign(Câu_trả_lời_chữ_thường=df_new['Câu trả lời'].str.lower())
#df_new = df_new.assign(Text_chữ_thường=df_new['Text'].str.lower())
def clean_text(text):
    text = unicodedata.normalize('NFC', text)
    text = text.lower()
    # Xóa các ký tự không phải chữ cái hoặc khoảng trắng, bao gồm dấu câu
    cleaned_text = re.sub(r'[^a-z0-9àáạảãâầấậẩẫăắằẵẳặđèẹẻéẽêềếệểễìíịỉĩỵỳýỷỹóọỏõòôồốộổỗơợởỡớờùúụủũưừứựửữ/\s]+', ' ', text)
    # Bỏ bớt khoảng trắng dư thừa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# Áp dụng hàm clean_text cho cột 'Câu trả lời'
df_new = df_new.assign(Câu_hỏi_sạch=df_new['Text'].apply(clean_text))
df_new = df_new.assign(Câu_trả_lời_sạch=df_new['Câu trả lời'].apply(clean_text))


In [5]:
df_new.head(3)

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch
0,Cá nhân đăng ký Bồi dưỡng nghiệp vụ đăng kiểm ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Theo quy định tại Khoản 3, Điều 8 Thông tư số ...","{'Tên': ['Cấp, cấp lại thẻ, dấu kỹ thuật đăng ...",Bộ Nông nghiệp và Phát triển nông thôn,cá nhân đăng ký bồi dưỡng nghiệp vụ đăng kiểm ...,theo quy định tại khoản 3 điều 8 thông tư số 2...
2,"Khi nào các tổ chức, cá nhân có thể gửi đề xuấ...",https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Định kỳ hàng năm theo thông báo trên trang thô...,{'Tên': ['Phê duyệt danh mục đề tài khoa học v...,Bộ Nông nghiệp và Phát triển nông thôn,khi nào các tổ chức cá nhân có thể gửi đề xuất...,định kỳ hàng năm theo thông báo trên trang thô...
3,Điều kiện công nhận đối với TBKT công nghệ sin...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Theo Điều 6 Thông tư số 23/2010/TT-BNNPTNT ngà...,{'Tên': ['Công nhận tiến bộ kỹ thuật công nghệ...,Bộ Nông nghiệp và Phát triển nông thôn,điều kiện công nhận đối với tbkt công nghệ sin...,theo điều 6 thông tư số 23/2010/tt bnnptnt ngà...


### Vấn đề 2:
#### Kiểm tra trùng hoàn toàn và xử lý (xét theo ministry)    
    + Xử lý các dòng trùng nhau hoàn toàn về câu hỏi và câu trả lời (ưu tiên câu trả lời có TTHC đầy đủ hơn, nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi)
    + Xử lý các dòng có câu hỏi trùng nhưng câu trả lời khác nhau ( ưu tiên câu trả lời có TTHC đầy đủ hơn, nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi)
Lý do không xử lý trường hợp câu hỏi khác nhau nhưng câu trả lời trùng là vì 1 câu trả lời có thể dành cho nhiều câu hỏi.

#### + Xử lý các dòng trùng nhau hoàn toàn về câu hỏi và câu trả lời (ưu tiên câu trả lời có TTHC đầy đủ hơn, nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi)

Đầu tiên xóa các dòng trùng hoàn toàn về câu hỏi, câu trả lời, TTHC

In [6]:
duplicates = df_new.duplicated(subset=['Câu_trả_lời_sạch','Câu_hỏi_sạch','TTHC', 'Ministry'],keep=False)
duplicate_rows_1 =df_new[duplicates]
df_new = df_new.drop_duplicates(subset=['Câu_trả_lời_sạch','Câu_hỏi_sạch','TTHC', 'Ministry'], keep='first')

Tiếp đó, Vì đã xóa những TTHC trùng nhau và chỉ giữ lại 1 cái, nên duplicate_row_2 không thể nào có 2 cái trùng nhau mà TTHC đều [] được, vì vậy ta drop [] của duplicate row 2. Tiến hành xử lý nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi

In [7]:
duplicates_2 = df_new.duplicated(subset=['Câu_trả_lời_sạch', 'Câu_hỏi_sạch', 'Ministry'],keep=False)
duplicate_rows_2 =df_new[duplicates_2]
df_new = df_new.drop(index=duplicate_rows_2.index)
duplicate_rows_2 = duplicate_rows_2[~duplicate_rows_2['TTHC'].str.contains(r'\[\]')]
index_to_drop = set()
for i in range(0, duplicate_rows_2.shape[0]-1):
    for j in range(i+1, duplicate_rows_2.shape[0]):
        if duplicate_rows_2.iloc[i].Câu_hỏi_sạch == duplicate_rows_2.iloc[j].Câu_hỏi_sạch:
            dict1 = ast.literal_eval(duplicate_rows_2.iloc[i].TTHC)
            dict2 = ast.literal_eval(duplicate_rows_2.iloc[j].TTHC)
            merged_dict = {'Tên': dict1['Tên'] + dict2['Tên'], 'Cơ quan thực hiện': [dict1['Cơ quan thực hiện'][0] + dict2['Cơ quan thực hiện'][0]]}
            duplicate_rows_2.iloc[i].TTHC = str(merged_dict)
            index_to_drop.add(duplicate_rows_2.index[j])
duplicate_rows_2 = duplicate_rows_2.drop(index=list(index_to_drop))
df_new = pd.concat([df_new, duplicate_rows_2])
df_new = df_new.reset_index(drop=True)

#### + Xử lý các dòng có câu hỏi trùng nhưng câu trả lời khác nhau ( ưu tiên câu trả lời có TTHC đầy đủ hơn, nếu cả 2 đều có số TTHC như nhau thì gộp lại thành 1 câu hỏi)

- BLEU ≈ 100%: Hai văn bản gần như giống hệt nhau.
- BLEU ≥ 70%: Hai văn bản rất tương tự nhau, có thể có một số khác biệt nhỏ như cách diễn đạt hoặc các từ đồng nghĩa.
- BLEU ≈ 50% - 70%: Hai văn bản có mức tương đồng đáng kể nhưng có thể có khác biệt ở cách diễn đạt, cấu trúc câu, hoặc từ vựng khác nhau.
- BLEU ≈ 30% - 50%: Hai văn bản khác nhau khá nhiều, nhưng vẫn có một số nội dung chung.
- BLEU < 30%: Hai văn bản khác nhau rõ rệt.

In [8]:
duplicates_3 = df_new.duplicated(subset=['Câu_hỏi_sạch','TTHC', 'Ministry'],keep=False)
duplicate_rows_3 =df_new[duplicates_3]
indexes_to_drop = set()
for i in range(0, duplicate_rows_3.shape[0]-1):
    if duplicate_rows_3.index[i] in indexes_to_drop:
        continue
    for j in range(i+1, duplicate_rows_3.shape[0]):
        if duplicate_rows_3.index[j] in indexes_to_drop:
            continue
        if duplicate_rows_3.iloc[i].Câu_hỏi_sạch == duplicate_rows_3.iloc[j].Câu_hỏi_sạch:
            references = duplicate_rows_3.iloc[i].Câu_trả_lời_sạch
            candidate = duplicate_rows_3.iloc[j].Câu_trả_lời_sạch
            bleu_score = sacrebleu.sentence_bleu(candidate, [references])
            #Nếu 2 câu trả lời có sự tương đồng cao
            if bleu_score.score > 50.0:
                if len(candidate.split()) > len(references.split()):
                    indexes_to_drop.add(duplicate_rows_3.index[i])
                else:
                    indexes_to_drop.add(duplicate_rows_3.index[j])
df_new = df_new.drop(index=list(indexes_to_drop))

Những câu còn thừa lại sau đây là những câu có bleu score thấp ( 2 câu trả lời gần như không liên quan tới nhau)

In [9]:
duplicates_4 = df_new.duplicated(subset=['Câu_hỏi_sạch','TTHC', 'Ministry'],keep=False)
duplicate_rows_4 =df_new[duplicates_4]
duplicate_rows_4

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch
2770,Người đăng ký dự thi có thể nộp hồ sơ tại cơ q...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Người đăng ký dự thi có thể nộp hồ sơ dự thi t...,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Tài chính,người đăng ký dự thi có thể nộp hồ sơ tại cơ q...,người đăng ký dự thi có thể nộp hồ sơ dự thi t...
2771,Người đăng ký dự thi có thể nộp hồ sơ tại cơ q...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"Người có đủ điều kiện, có nguyện vọng dự thi c...","{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Tài chính,người đăng ký dự thi có thể nộp hồ sơ tại cơ q...,người có đủ điều kiện có nguyện vọng dự thi cấ...
3294,Số lượng hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,01 bộ bản chính,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,số lượng hồ sơ,01 bộ bản chính
3295,Số lượng hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,01 bộ,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,số lượng hồ sơ,01 bộ
3296,Số lượng hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"02 bộ (01 bộ bản chính, 01 bộ bản sao).","{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,số lượng hồ sơ,02 bộ 01 bộ bản chính 01 bộ bản sao
3400,Thành phần hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,.Hồ sơ thông báo thay đổi thay đổi tên miền kh...,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,thành phần hồ sơ,hồ sơ thông báo thay đổi thay đổi tên miền khi...
3401,Thành phần hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Hồ sơ đề nghị cấp giấy phép thiết lập trang th...,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,thành phần hồ sơ,hồ sơ đề nghị cấp giấy phép thiết lập trang th...
3402,Thành phần hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,"- Văn bản đề nghị (nêu rõ nội dung, lý do cần ...","{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,thành phần hồ sơ,văn bản đề nghị nêu rõ nội dung lý do cần sửa ...
3403,Thành phần hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Hồ sơ thông báo thay đổi chủ sở hữu; địa chỉ t...,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,thành phần hồ sơ,hồ sơ thông báo thay đổi chủ sở hữu địa chỉ tr...
3405,Thành phần hồ sơ,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Hồ sơ đề nghị cấp giấy phép cung cấp dịch vụ t...,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Thông tin và Truyền thông,thành phần hồ sơ,hồ sơ đề nghị cấp giấy phép cung cấp dịch vụ t...


Theo quan sát thu được thì các câu hỏi 'Số lượng hồ sơ', 'Thành phần hồ sơ' khá tối nghĩa, không chỉ rõ mục đích câu hỏi => drop

In [10]:
cau_hoi_toi_nghia = ['số lượng hồ sơ', 'thành phần hồ sơ', 'thời hạn giải quyết hồ sơ','thời hạn giải quyết là bao lâu','thủ tục cấp lại cần những gì']
duplicate_rows_4 = duplicate_rows_4[duplicate_rows_4['Câu_hỏi_sạch'].isin(cau_hoi_toi_nghia)]
df_new = df_new.drop(index=duplicate_rows_4.index)
df_new = df_new.reset_index(drop=True)

In [11]:
duplicates_5 = df_new.duplicated(subset=['Câu_hỏi_sạch', 'Ministry'],keep=False)
duplicate_rows_5 =df_new[duplicates_5]
df_new = df_new.drop(index=duplicate_rows_5.index)
duplicate_rows_5 = duplicate_rows_5[~duplicate_rows_5['TTHC'].str.contains(r'\[\]')]
index_to_drop = set()
for i in range(duplicate_rows_5.shape[0]-1):
    for j in range(i+1, duplicate_rows_5.shape[0]):
        if duplicate_rows_5.iloc[i].Câu_hỏi_sạch == duplicate_rows_5.iloc[j].Câu_hỏi_sạch :
            references = duplicate_rows_5.iloc[i].Câu_trả_lời_sạch
            candidate = duplicate_rows_5.iloc[j].Câu_trả_lời_sạch
            bleu_score = sacrebleu.sentence_bleu(candidate, [references])
            if bleu_score.score > 50.0:
                dict1 = ast.literal_eval(duplicate_rows_5.iloc[i].TTHC)
                dict2 = ast.literal_eval(duplicate_rows_5.iloc[j].TTHC)
                merged_dict = {'Tên': dict1['Tên'] + dict2['Tên'], 'Cơ quan thực hiện': [dict1['Cơ quan thực hiện'][0] + dict2['Cơ quan thực hiện'][0]]}
                duplicate_rows_5.iloc[i].TTHC = str(merged_dict)
                index_to_drop.add(duplicate_rows_5.index[j])
duplicate_rows_5 = duplicate_rows_5.drop(index=list(index_to_drop))
df_new = pd.concat([df_new, duplicate_rows_5])
df_new = df_new.reset_index(drop=True)

In [12]:
#df_bleu_index = set()
#for i in range(df_new.shape[0]-1):
    #if df_new.index[i] in df_bleu_index:
        #continue
    #for j in range(i+1, df_new.shape[0]):
        #if df_new.index[j] in df_bleu_index:
            #continue
        #else:
            #references = df_new.iloc[i].Câu_hỏi_sạch
            #candidate = df_new.iloc[j].Câu_hỏi_sạch
            #bleu_score = sacrebleu.sentence_bleu(candidate, [references])
            #if bleu_score.score > 70.0:
                #print(i,j)
                #df_bleu_index.add(df_new.index[i])
                #df_bleu_index.add(df_new.index[j])

In [13]:
#df_row = df_new.loc[list(df_bleu_index)]
#df_row

In [14]:
df_new.describe()

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch
count,6861,6861,6861,6861,6861,6861,6861
unique,6858,6861,6619,1720,16,6857,6587
top,Tổ chức được cấp giấy phép thử nghiệm mạng và ...,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,Không mất phí.,"{'Tên': [], 'Cơ quan thực hiện': []}",Bộ Nông nghiệp và Phát triển nông thôn,tổ chức được cấp giấy phép thử nghiệm mạng và ...,không mất phí
freq,2,1,24,2447,813,2,26


convert TTHC to dict

Divide TTHC into Tên_TTHC and Tên_cơ_quan_thực_hiện

In [15]:
df_new['TTHC'] = df_new['TTHC'].apply(ast.literal_eval)
df_new['Tên_TTHC'] = df_new['TTHC'].apply(lambda x: x['Tên'] if isinstance(x, dict) and 'Tên' in x else None)
df_new['Tên_cơ_quan_thực_hiện'] = df_new['TTHC'].apply(lambda x: x['Cơ quan thực hiện'] if isinstance(x, dict) and 'Cơ quan thực hiện' in x else None)

In [16]:
df_nanNameTTHC = df_new[df_new['Tên_TTHC'].apply(lambda x: x == [])]
df_nanAgencyTTHC = df_new[df_new.Tên_cơ_quan_thực_hiện.apply(lambda x: x == [])]
df_nanAgencyTTHC.shape
if df_nanAgencyTTHC.equals(df_nanNameTTHC):
    print('true')
else:
    print('false')

true


In [17]:
df_new[df_new['Câu trả lời'].isnull() | (df_new['Câu trả lời'] == '')]

,Text,Link,Câu trả lời,TTHC,Ministry,Câu_hỏi_sạch,Câu_trả_lời_sạch,Tên_TTHC,Tên_cơ_quan_thực_hiện


In [18]:
agencies = []
for i in range(df_new.shape[0]):
    for item in df_new.TTHC[i]['Cơ quan thực hiện']:
        for sub_item in item:
            if sub_item not in agencies:
                agencies.append(sub_item)
                

In [19]:
len(agencies)

387

In [20]:
TTHC = []
for i in range(df_new.shape[0]):
    for item in df_new.TTHC[i]['Tên']: 
        if item not in TTHC:
            print(item)
            TTHC.append(item)
                

Cấp, cấp lại thẻ, dấu kỹ thuật đăng kiểm viên tàu cá
Phê duyệt danh mục đề tài khoa học và công nghệ, dự án sản xuất thử nghiệm cấp Bộ
Công nhận tiến bộ kỹ thuật công nghệ sinh học
Cấp, cấp lại giấy chứng nhận cơ sở đủ điều kiện đóng mới, cải hoán tàu cá
Xác nhận cam kết hoặc chứng nhận sản phẩm thủy sản xuất khẩu có nguồn gốc từ thủy sản khai thác nhập khẩu (theo yêu cầu)
Cấp, cấp lại, gia hạn giấy phép hoạt động cho tổ chức, cá nhân nước ngoài có tàu hoạt động thuỷ sản trong vùng biển Việt Nam
Tuyển chọn, giao trực tiếp tổ chức và cá nhân chủ trì đề tài khoa học và công nghệ, dự án sản xuất thử nghiệm cấp bộ
Cấp văn bản chấp thuận cho tàu cá khai thác thuỷ sản ở vùng biển ngoài vùng biển Việt Nam hoặc cấp phép cho đi khai thác tại vùng biển thuộc thẩm quyền quản lý của Tổ chức nghề cá khu vực
Cấp, cấp lại giấy phép khai thác thủy sản
Điều chỉnh, đề tài khoa học và công nghệ, dự án sản xuất thử nghiệm cấp bộ
Công bố mở cảng cá loại 3
Hỗ trợ một lần sau đầu tư đóng mới tàu cá
Đánh giá 

In [21]:
len(TTHC) #cua df 

1924

In [22]:
df_new.to_csv('Preprocessed Data.csv', index=False, encoding='utf-8-sig')